In [500]:
import pandas as pd
import fonctions
pd.options.display.max_columns = 150
pd.options.display.max_rows = 150

In [501]:
df_objet = pd.read_sql_table('Objet', 'sqlite:///db.sqlite')
df_gare = pd.read_sql_table('Gare', 'sqlite:///db.sqlite')
df_meteo = pd.read_sql_table('Meteo', 'sqlite:///db.sqlite')
df_freq = pd.read_sql_table('Frequence', 'sqlite:///db.sqlite')

In [502]:
# On définis les dates en index pour les utiliser plus facilement

df_objet['date'] = pd.to_datetime(df_objet['date'])
df_objet.set_index('date', inplace=True)

In [503]:
question1 = df_objet.groupby([pd.Grouper(freq = 'w')]).agg(nb_objets = ('type' , 'count'))

In [504]:
question1 = question1.reset_index()

In [505]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.histogram(question1, y="nb_objets" ,x="date",labels={'date':'Date', 'nb_objets':'Objets perdu'}, nbins=200)
fig.update_layout(bargap=0.01)
fig.show()

In [506]:
question2 = df_objet.groupby([pd.Grouper(freq = 'w'), "type"]).agg(nb_objets = ('type' , 'count')).reset_index()

In [507]:
fig = px.area(question2, y="nb_objets" ,x="date", color="type", labels={'date':'Date', 'nb_objets':'Objets perdu'})
fig.update_layout(bargap=0.01)
fig.show()

In [508]:
# On importe notre fichier avec nos fonctions
import fonctions

# On drop les index ou il nous manquent les adresse_cp
df_gare = df_gare.dropna()

# On transforme nos régions pour qu'elles correspondent au format actuel
df_gare['gare_regionsncf_libelle'] = df_gare['adresse_cp'].apply(lambda x: fonctions.search_region(x[:2]))

In [509]:
# On tranforme nos codes UIC pour qu'ils correspondent au format des autres df
df_freq['code_uic_complet'] = df_freq['code_uic_complet'].apply(lambda x: "00"+str(x))
df_freq['code_uic_complet']

0       0087313759
1       0087481614
2       0087317362
3       0087545269
4       0087386052
           ...    
2965    0087645101
2966    0087191304
2967    0087413385
2968    0087193649
2969    0087182196
Name: code_uic_complet, Length: 2970, dtype: object

In [510]:
df_objet = df_objet.reset_index()

In [511]:
question3 = pd.merge(left=df_objet, right=df_gare, how="left",left_on="gare_code_uic", right_on="uic_code")
question3 = question3.dropna()

In [530]:
question3

,date,recordid,gare_code_uic,type,nature,recordtype,uic_code,departement_libellemin,gare_regionsncf_libelle,gare_alias_libelle_noncontraint,adresse_cp,code_gare,lat,long
3,2016-01-06 07:55:24+00:00,90432de5460f71c38de83362302235db545f018b,0087746008,Optique,Lunettes en étui,Objet trouvé,0087746008,Haute-Savoie,Auvergne-Rhône-Alpes,Annecy,74000,00357,45.901965,6.121835
4,2016-01-06 08:32:13+00:00,c76a2ff3c986c9748db61f07687580ecb0af4b5a,0087722025,"Vélos, trottinettes, accessoires 2 roues","Casque (vélo, moto)",Objet trouvé,0087722025,Rhône,Auvergne-Rhône-Alpes,Lyon Perrache,69286,00553,45.748476,4.825733
5,2016-01-06 08:33:19+00:00,cf34554fbe00a0ef4fc71ee154020cac5fad2452,0087317586,"Appareils électroniques, informatiques, appare...",Téléphone portable,Objet trouvé,0087317586,Pas-de-Calais,Hauts-de-France,Boulogne-sur-Mer,62200,02210,50.715700,1.609640
6,2016-01-06 08:34:26+00:00,eb010515dbb542ad116fe9bce680231818cdccb3,0087722025,"Bagagerie: sacs, valises, cartables","Sac d'enseigne (plastique, papier, …)",Objet trouvé,0087722025,Rhône,Auvergne-Rhône-Alpes,Lyon Perrache,69286,00553,45.748476,4.825733
8,2016-01-06 08:40:26+00:00,48ddd8bf36b90301c0a89f111004934fd5799281,0087722025,"Bagagerie: sacs, valises, cartables","Sac d'enseigne (plastique, papier, …)",Objet trouvé,0087722025,Rhône,Auvergne-Rhône-Alpes,Lyon Perrache,69286,00553,45.748476,4.825733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481294,2021-12-31 16:11:03+00:00,ad6947926569cab29d0a06b0424bd2eb9e86274a,0087271494,"Bagagerie: sacs, valises, cartables","Valise, sac sur roulettes",Objet trouvé,0087271494,Val-d'Oise,Île-de-France,Aéroport Charles de Gaulle 2 TGV,95716,02541,49.003652,2.570892
481297,2021-12-31 16:35:25+00:00,fee5e9002e275fc65800701143912b8d6e37e211,0087686006,"Appareils électroniques, informatiques, appare...",Téléphone portable,Objet trouvé,0087686006,Paris,Île-de-France,Paris Gare de Lyon,75012,00753,48.844888,2.373520
481298,2021-12-31 16:59:39+00:00,ef1f506126b1f9c11ef5bad8bd08666fa7c0e970,0087686006,"Bagagerie: sacs, valises, cartables",Sac à dos,Objet trouvé,0087686006,Paris,Île-de-France,Paris Gare de Lyon,75012,00753,48.844888,2.373520
481299,2021-12-31 17:02:32+00:00,9640315e51341e023ca6cebf39e762909cf56780,0087773002,"Porte-monnaie / portefeuille, argent, titres",Carte de crédit,Objet trouvé,0087773002,Hérault,Occitanie,Montpellier Saint-Roch,34011,00130,43.604738,3.880674


In [608]:
all_data = pd.merge(left=question3, right=df_freq, how="inner", left_on="uic_code", right_on="code_uic_complet")
all_data.drop_duplicates(subset=['recordid_x'])
all_data = all_data.drop(columns=["code_uic_complet", "gare_code_uic", "recordid_x", "recordid_y", "region_name", "gare_alias_libelle_noncontraint"], axis=1)

In [609]:
all_data['date'] = pd.to_datetime(all_data['date'])
all_data.set_index('date', inplace=True)

In [610]:
group_y = all_data.groupby([pd.Grouper(freq = 'y'),'gare_regionsncf_libelle','total_voyageurs_2016','total_voyageurs_2017', 'total_voyageurs_2018', 'total_voyageurs_2019','total_voyageurs_2020', 'total_voyageurs_2021','type' ]).agg(nb_objets = ('type' , 'count')).reset_index()

In [611]:
group_y['date'] = pd.to_datetime(group_y['date'])
group_y.set_index('date', inplace=True)

In [612]:
group_y = group_y.reset_index()

In [622]:
test = pd.melt(group_y, id_vars=['gare_regionsncf_libelle','type','nb_objets'], var_name="date",value_name="freq",value_vars=['total_voyageurs_2016','total_voyageurs_2017', 'total_voyageurs_2018', 'total_voyageurs_2019','total_voyageurs_2020', 'total_voyageurs_2021'])
test['date'] = test['date'].apply(lambda x: x[16:])

In [623]:
test

,gare_regionsncf_libelle,type,nb_objets,date,freq
0,Auvergne-Rhône-Alpes,"Appareils électroniques, informatiques, appare...",3,2016,173811
1,Auvergne-Rhône-Alpes,"Articles d'enfants, de puériculture",1,2016,173811
2,Auvergne-Rhône-Alpes,"Articles de sport, loisirs, camping",3,2016,173811
3,Auvergne-Rhône-Alpes,"Bagagerie: sacs, valises, cartables",12,2016,173811
4,Auvergne-Rhône-Alpes,"Clés, porte-clés, badge magnétique",1,2016,173811
...,...,...,...,...,...
45001,Île-de-France,Parapluies,23,2021,79986275
45002,Île-de-France,Pièces d'identités et papiers personnels,500,2021,79986275
45003,Île-de-France,"Porte-monnaie / portefeuille, argent, titres",778,2021,79986275
45004,Île-de-France,"Vélos, trottinettes, accessoires 2 roues",95,2021,79986275


In [ ]:
test['date'] = pd.to_datetime(test['date'])
test.set_index('date', inplace=True)

In [666]:
test = test.reset_index()

In [672]:
obj_y_r = test.groupby(["gare_regionsncf_libelle", "type"])

In [673]:
obj_y_r.head()

,date,gare_regionsncf_libelle,type,nb_objets,freq
0,2016-01-01,Auvergne-Rhône-Alpes,"Appareils électroniques, informatiques, appare...",3,173811
1,2016-01-01,Auvergne-Rhône-Alpes,"Articles d'enfants, de puériculture",1,173811
2,2016-01-01,Auvergne-Rhône-Alpes,"Articles de sport, loisirs, camping",3,173811
3,2016-01-01,Auvergne-Rhône-Alpes,"Bagagerie: sacs, valises, cartables",12,173811
4,2016-01-01,Auvergne-Rhône-Alpes,"Clés, porte-clés, badge magnétique",1,173811
...,...,...,...,...,...
2519,2016-01-01,Provence-Alpes-Côte d'Azur,Parapluies,1,3403078
2988,2016-01-01,Bretagne,Articles médicaux,8,1631603
3627,2016-01-01,Occitanie,Parapluies,1,0
3683,2016-01-01,Occitanie,Parapluies,1,1508061


In [680]:
df_meteo_obj = pd.read_csv('CSV/meteo_obj.csv')

In [681]:
df_meteo_obj = df_meteo_obj.drop('Unnamed: 0', axis=1)

In [683]:
df_meteo_obj.columns

Index(['temperature_moyenne', 'nb_lost_item'], dtype='object')

In [687]:
fig = px.scatter(x=df_meteo_obj['temperature_moyenne'], y=df_meteo_obj['nb_lost_item'], labels={'y':'Nb objets perdu', 'x':'Temperature (°C)'})
fig.show()

In [690]:
df_meteo_obj.corr()

,temperature_moyenne,nb_lost_item
temperature_moyenne,1.000000,0.077594
nb_lost_item,0.077594,1.000000
